In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

In [ ]:
#| default_exp processing

In [ ]:
#| export
from typing import Optional, Tuple

import numpy as np
import pandas as pd

from utilsforecast.compat import DataFrame, Series, pl_Series
from utilsforecast.grouped_array import GroupedArray
from utilsforecast.validation import validate_format

In [ ]:
#| exporti
def _polars_categorical_to_numerical(serie: pl_Series) -> pl_Series:
    import polars as pl

    if serie.dtype == pl.Categorical:
        serie = serie.cast(pl.Utf8).cast(pl.Float64)
    return serie

In [ ]:
#| export
class DataFrameProcessor:
    def __init__(
        self,
        id_col: str = 'unique_id',
        time_col: str = 'ds',
        target_col: str = 'y',
    ):
        """Class to  extract common structures from pandas and polars dataframes.
        
        Parameters
        ----------
        id_col : str (default='unique_id')
            Column that identifies each serie.
        time_col : str (default='ds')
            Column that identifies each timestep, its values can be timestamps or integers.
        target_col : str (default='y')
            Column that contains the target.
        """
        self.id_col = id_col
        self.time_col = time_col
        self.target_col = target_col

    def _counts_by_id(self, df: DataFrame) -> DataFrame:
        if isinstance(df, pd.DataFrame):
            id_counts = df.groupby(self.id_col, observed=True).size()
            id_counts = id_counts.reset_index()
            id_counts.columns = [self.id_col, 'counts']
        else:
            id_counts = df[self.id_col].value_counts().sort(self.id_col)
        return id_counts

    def _value_cols_to_numpy(self, df: DataFrame) -> np.ndarray:
        exclude_cols = [self.id_col, self.time_col, self.target_col]
        value_cols = [col for col in df.columns if col not in exclude_cols]
        # ensure target is the first column
        value_cols = [self.target_col] + value_cols
        if isinstance(df, pd.DataFrame):
            data = df[value_cols].to_numpy()
        else:
            import polars as pl

            try:
                expr = pl.all().map_batches(_polars_categorical_to_numerical)
            except AttributeError:
                expr = pl.all().map(_polars_categorical_to_numerical)
    
            data = df[value_cols].select(expr).to_numpy()
        return data

    def maybe_compute_sort_indices(self, df: DataFrame) -> Optional[np.ndarray]:
        """Compute indices that would sort dataframe
        
        Parameters
        ----------
        df : pandas or polars DataFrame
            Input dataframe with id, times and target values.
    
        Returns
        -------
        numpy array or None
            Array with indices to sort the dataframe or None if it's already sorted.
        """        
        if isinstance(df, pd.DataFrame):
            idx = pd.MultiIndex.from_arrays([df[self.id_col], df[self.time_col]])
        else:
            import polars as pl

            sort_idxs = df.select(
                pl.arg_sort_by([self.id_col, self.time_col]).alias("idx")
            )['idx'].to_numpy()
            idx = pd.Index(sort_idxs)
        if idx.is_monotonic_increasing:
            return None
        if isinstance(df, pd.DataFrame):
            sort_idxs = idx.argsort()
        return sort_idxs    

    def process(
        self,
        df: DataFrame
    ) -> Tuple[Series, np.ndarray, GroupedArray, Optional[np.ndarray]]:
        """Extract components from dataframe
        
        Parameters
        ----------
        df : pandas or polars DataFrame
            Input dataframe with id, times and target values.
    
        Returns
        -------
        ids : pandas or polars Serie
            serie with the sorted unique ids present in the data.
        last_times : numpy array
            array with the last time for each serie.
        sort_idxs : numpy array or None
            array with the indices that would sort the original data.
            If the data is already sorted this is `None`.
        grouped_array : GroupedArray
            grouped array structure with the data split and sorted by serie.
        """
        # validations
        validate_format(df, self.id_col, self.time_col, self.target_col)

        # ids
        id_counts = self._counts_by_id(df)
        uids = id_counts[self.id_col]

        # indices
        indptr = np.append(
            np.int64(0),
            id_counts['counts'].to_numpy().cumsum().astype(np.int64),
        )
        last_idxs = indptr[1:] - 1        

        # data
        data = self._value_cols_to_numpy(df)
        # ensure float dtype
        if data.dtype not in (np.float32, np.float64):
            data = data.astype(np.float32)
        # ensure 2d
        if data.ndim == 1:
            data = data.reshape(-1, 1)

        # check if we need to sort
        sort_idxs = self.maybe_compute_sort_indices(df)
        if sort_idxs is not None:
            data = data[sort_idxs]
            last_idxs = sort_idxs[last_idxs]
        ga = GroupedArray(data, indptr)
        times = df[self.time_col].to_numpy()[last_idxs]
        return uids, times, ga, sort_idxs

In [ ]:
from nbdev import show_doc

In [ ]:
show_doc(DataFrameProcessor)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/processing.py#L25){target="_blank" style="float:right; font-size:smaller"}

### DataFrameProcessor

>      DataFrameProcessor (id_col:str='unique_id', time_col:str='ds',
>                          target_col:str='y')

Class to  extract common structures from pandas and polars dataframes.

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| id_col | str | unique_id | Column that identifies each serie. |
| time_col | str | ds | Column that identifies each timestep, its values can be timestamps or integers. |
| target_col | str | y | Column that contains the target. |

In [ ]:
show_doc(DataFrameProcessor.process)

---

[source](https://github.com/Nixtla/utilsforecast/blob/main/utilsforecast/processing.py#L102){target="_blank" style="float:right; font-size:smaller"}

### DataFrameProcessor.process

>      DataFrameProcessor.process
>                                  (df:Union[pandas.core.frame.DataFrame,polars.
>                                  dataframe.frame.DataFrame])

Extract components from dataframe

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| df | Union | Input dataframe with id, times and target values. |
| **Returns** | **Tuple** | **serie with the sorted unique ids present in the data.** |

In [ ]:
import polars as pl
from fastcore.test import test_eq

from utilsforecast.data import generate_series

In [ ]:
static_features = ['static_0', 'static_1']

In [ ]:
for n_static_features in [0, 2]:
    series_pd = generate_series(1_000, n_static_features=n_static_features, equal_ends=False, engine='pandas')
    scrambled_series_pd = series_pd.sample(frac=1.0)
    dfp = DataFrameProcessor('unique_id', 'ds', 'y')
    uids, times, ga, _ = dfp.process(scrambled_series_pd)
    test_eq(times, series_pd.groupby('unique_id', observed=True)['ds'].max().values)
    test_eq(uids, np.sort(series_pd['unique_id'].unique()))
    test_eq(ga.data, series_pd[['y'] + static_features[:n_static_features]].to_numpy())
    test_eq(np.diff(ga.indptr), series_pd.groupby('unique_id', observed=True).size().values)

In [ ]:
for n_static_features in [0, 2]:
    series_pl = generate_series(1_000, n_static_features=n_static_features, equal_ends=False, engine='polars')
    scrambled_series_pl = series_pl.sample(fraction=1.0, shuffle=True)
    dfp = DataFrameProcessor('unique_id', 'ds', 'y')
    uids, times, ga, sort_idxs = dfp.process(scrambled_series_pl)
    try:
        grouped = series_pl.group_by('unique_id')
    except AttributeError:
        grouped = series_pl.groupby('unique_id')
    test_eq(times, grouped.agg(pl.col('ds').max()).sort('unique_id')['ds'].to_numpy())
    test_eq(uids, series_pl['unique_id'].unique().sort())
    test_eq(ga.data, series_pl.select(['y'] + static_features[:n_static_features]).to_numpy().astype('float64'))
    test_eq(np.diff(ga.indptr), grouped.count().sort('unique_id')['count'].to_numpy())